In [1]:
! ls ../input/humpback-whale-identification

sample_submission.csv  test  train  train.csv


In [2]:
! head -n 5 ../input/humpback-whale-identification/train.csv

Image,Id
0000e88ab.jpg,w_f48451c
0001f9222.jpg,w_c3d896a
00029d126.jpg,w_20df2c5
00050a15a.jpg,new_whale


In [3]:
! rm -rf ./train
! mkdir ./train

In [4]:
import os
import shutil
import csv

known_ids = {}
with open('../input/humpback-whale-identification/train.csv', 'r') as infile:
    rdr = csv.reader(infile)
    for row in rdr:
        img = row[0]
        id = row[1]
        if id == 'Id':
            continue
        if id not in known_ids:
            os.mkdir('./train/' + id)
            known_ids[id] = True
        shutil.copy('../input/humpback-whale-identification/train/' + img, './train/' + id + '/' + img)

In [5]:
! ls ../input/timmmaster

LICENSE		      docs			   requirements.txt
MANIFEST.in	      hubconf.py		   results
README.md	      inference.py		   setup.cfg
avg_checkpoints.py    mkdocs.yml		   setup.py
benchmark.py	      model-index.yml		   tests
clean_checkpoint.py   notebooks			   timm
convert		      requirements-docs.txt	   train.py
distributed_train.sh  requirements-modelindex.txt  validate.py


In [6]:
# based on https://www.kaggle.com/piantic/how-to-finetuning-models-pytorch-xla-tpu
import sys
sys.path.append('../input/timmmaster')
import timm

In [7]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
import numpy as np

In [8]:
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.RandomResizedCrop(256, [0.8, 1.2]),
    transforms.ToTensor()
])

allimg = datasets.ImageFolder('./train', transform=transform)

validation_split = 0.15
dataset_size = len(allimg)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.seed(101)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [9]:
train_loader = torch.utils.data.DataLoader(allimg, batch_size=16, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(allimg, batch_size=16,
                                                sampler=valid_sampler)

In [10]:
from torch import nn

class CustomResNext(nn.Module):
    def __init__(self, model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, target_size)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.fc.parameters():
            param.requires_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True

In [11]:
allimg.class_to_idx

{'new_whale': 0,
 'w_0003639': 1,
 'w_0003c59': 2,
 'w_0027efa': 3,
 'w_00289b1': 4,
 'w_002c810': 5,
 'w_0032a46': 6,
 'w_003bae6': 7,
 'w_00656c0': 8,
 'w_0066399': 9,
 'w_007fefa': 10,
 'w_00904a7': 11,
 'w_009c9c5': 12,
 'w_00a41ba': 13,
 'w_00b3dc2': 14,
 'w_00d50c9': 15,
 'w_00d5466': 16,
 'w_00d5e98': 17,
 'w_00f340d': 18,
 'w_010f858': 19,
 'w_0115c24': 20,
 'w_0118bab': 21,
 'w_012678c': 22,
 'w_0135f5f': 23,
 'w_0140c92': 24,
 'w_015a4bf': 25,
 'w_016f0ea': 26,
 'w_0182687': 27,
 'w_0189b6d': 28,
 'w_01976db': 29,
 'w_01a1d88': 30,
 'w_01cb0c4': 31,
 'w_01cbbbd': 32,
 'w_01d790e': 33,
 'w_01e1b97': 34,
 'w_01e1fe0': 35,
 'w_01ed442': 36,
 'w_01f14e1': 37,
 'w_01fc429': 38,
 'w_02156b0': 39,
 'w_022b708': 40,
 'w_022d2f5': 41,
 'w_02469a1': 42,
 'w_02545ea': 43,
 'w_025911c': 44,
 'w_0277a07': 45,
 'w_027f528': 46,
 'w_0292e15': 47,
 'w_02aa597': 48,
 'w_02c7e9d': 49,
 'w_02d138d': 50,
 'w_02d5c46': 51,
 'w_02e8199': 52,
 'w_02ff6d6': 53,
 'w_0301302': 54,
 'w_030ac9b': 55,
 '

In [12]:
m = CustomResNext('resnext50_32x4d', len(allimg.class_to_idx.keys()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnext50_32x4d_a1h-0146ab0a.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_a1h-0146ab0a.pth


CustomResNext(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [13]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

class LabelSmoothingLoss(nn.Module): 
    def __init__(self, classes=5, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad():
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
        
class TaylorSoftmax(nn.Module):
    '''
    This is the autograd version
    '''
    def __init__(self, dim=1, n=2):
        super(TaylorSoftmax, self).__init__()
        assert n % 2 == 0
        self.dim = dim
        self.n = n

    def forward(self, x):
        '''
        usage similar to nn.Softmax:
            >>> mod = TaylorSoftmax(dim=1, n=4)
            >>> inten = torch.randn(1, 32, 64, 64)
            >>> out = mod(inten)
        '''
        fn = torch.ones_like(x)
        denor = 1.
        for i in range(1, self.n+1):
            denor *= i
            fn = fn + x.pow(i) / denor
        out = fn / fn.sum(dim=self.dim, keepdims=True)
        return out

In [14]:
! pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git > lol

  Running command git clone --filter=blob:none -q https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-_fs8i6k6


In [15]:
from warmup_scheduler import GradualWarmupScheduler
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

class SymmetricCrossEntropy(nn.Module):

    def __init__(self, alpha=0.1, beta=1.0, num_classes=5):
        super(SymmetricCrossEntropy, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.num_classes = num_classes

    def forward(self, logits, targets, reduction='mean'):
        onehot_targets = torch.eye(self.num_classes)[targets].cuda()
        ce_loss = F.cross_entropy(logits, targets, reduction=reduction)
        rce_loss = (-onehot_targets*logits.softmax(1).clamp(1e-7, 1.0).log()).sum(1)
        if reduction == 'mean':
            rce_loss = rce_loss.mean()
        elif reduction == 'sum':
            rce_loss = rce_loss.sum()
        return self.alpha * ce_loss + self.beta * rce_loss

In [16]:
from torch.optim import Adam
#from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=1e-4, weight_decay=1e-6, amsgrad=False)

scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 3)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
scheduler = scheduler_warmup
criterion = FocalLoss().to(device)
#criterion = SymmetricCrossEntropy().to(device)
#TaylorCrossEntropyLoss(smoothing=0.05).to(device)
best_score = 0.
best_loss = np.inf
config_device = "GPU"

In [17]:
import math

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [18]:
from torch.cuda.amp import autocast, GradScaler

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            # record loss
            losses.update(loss.item(), batch_size)
#             if CFG.gradient_accumulation_steps > 1:
#                 loss = loss / CFG.gradient_accumulation_steps
            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1000)
            if (step + 1) % 1 == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 100 == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    trues = []
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        trues.append(labels.to('cpu').numpy())
        preds.append(y_preds.softmax(1).to('cpu').numpy())
#         if CFG.gradient_accumulation_steps > 1:
#             loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % 100 == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    trues = np.concatenate(trues)
    predictions = np.concatenate(preds)
    return losses.avg, predictions, trues

In [19]:
from sklearn.metrics import accuracy_score
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [20]:
valid_labels = []
for img, labels in validation_loader:
    valid_labels += labels.tolist()
valid_labels[:20]

[4573,
 0,
 0,
 2728,
 4608,
 0,
 1194,
 0,
 0,
 253,
 3604,
 4713,
 3468,
 4935,
 0,
 0,
 0,
 1168,
 64,
 0]

In [21]:
import time
for epoch in range(5):

    start_time = time.time()

    avg_loss = train_fn(train_loader, m, criterion, optimizer, epoch, scheduler, device)
    avg_val_loss, preds, _ = valid_fn(validation_loader, m, criterion, device)
    
#     if isinstance(scheduler, ReduceLROnPlateau):
#         scheduler.step(avg_val_loss)
#     elif isinstance(scheduler, CosineAnnealingLR):
#         scheduler.step()
#     elif isinstance(scheduler, CosineAnnealingWarmRestarts):
#         scheduler.step()
#     el
    if isinstance(scheduler, GradualWarmupSchedulerV2):
        scheduler.step(epoch)

    score = get_score(valid_labels, preds.argmax(1))

    elapsed = time.time() - start_time

    print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    print(f'Epoch {epoch+1} - Score: {score:.4f}')
    
    if score > best_score:
        best_score = score
        print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        torch.save({'model': m.state_dict(), 
                        'preds': preds},
                       'best_score.pth')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [1][0/1348] Data 0.314 (0.314) Elapsed 0m 6s (remain 143m 15s) Loss: 8.4882(8.4882) Grad: nan  
Epoch: [1][100/1348] Data 0.311 (0.317) Elapsed 0m 59s (remain 12m 11s) Loss: 6.0186(7.6575) Grad: 65433.7266  
Epoch: [1][200/1348] Data 0.306 (0.319) Elapsed 1m 52s (remain 10m 42s) Loss: 5.1253(6.9178) Grad: 55036.4609  
Epoch: [1][300/1348] Data 0.289 (0.315) Elapsed 2m 44s (remain 9m 32s) Loss: 3.8588(6.4916) Grad: 78943.3125  
Epoch: [1][400/1348] Data 0.285 (0.315) Elapsed 3m 37s (remain 8m 33s) Loss: 5.2456(6.3271) Grad: 35148.0234  
Epoch: [1][500/1348] Data 0.469 (0.316) Elapsed 4m 30s (remain 7m 37s) Loss: 3.8123(6.2275) Grad: 22299.3516  
Epoch: [1][600/1348] Data 0.335 (0.316) Elapsed 5m 22s (remain 6m 41s) Loss: 5.1759(6.1771) Grad: 43647.6914  
Epoch: [1][700/1348] Data 0.324 (0.317) Elapsed 6m 16s (remain 5m 47s) Loss: 4.9482(6.1520) Grad: 10811.5928  
Epoch: [1][800/1348] Data 0.329 (0.316) Elapsed 7m 8s (remain 4m 52s) Loss: 7.0412(6.1358) Grad: 10155.7812  
Epoch: [

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [2][0/1348] Data 0.312 (0.312) Elapsed 0m 0s (remain 11m 49s) Loss: 5.9058(5.9058) Grad: inf  
Epoch: [2][100/1348] Data 0.292 (0.323) Elapsed 0m 54s (remain 11m 7s) Loss: 6.6177(5.2394) Grad: 92160.7188  
Epoch: [2][200/1348] Data 0.294 (0.314) Elapsed 1m 45s (remain 10m 3s) Loss: 4.8330(5.3297) Grad: 61104.9141  
Epoch: [2][300/1348] Data 0.263 (0.313) Elapsed 2m 38s (remain 9m 9s) Loss: 5.9034(5.2698) Grad: 89850.5469  
Epoch: [2][400/1348] Data 0.287 (0.312) Elapsed 3m 30s (remain 8m 16s) Loss: 5.5849(5.2967) Grad: 63200.7461  
Epoch: [2][500/1348] Data 0.302 (0.311) Elapsed 4m 22s (remain 7m 23s) Loss: 4.3493(5.3048) Grad: 44264.0703  
Epoch: [2][600/1348] Data 0.288 (0.311) Elapsed 5m 14s (remain 6m 30s) Loss: 4.9935(5.2748) Grad: 45702.1250  
Epoch: [2][700/1348] Data 0.266 (0.310) Elapsed 6m 6s (remain 5m 37s) Loss: 3.4004(5.2699) Grad: 31859.1855  
Epoch: [2][800/1348] Data 0.285 (0.310) Elapsed 6m 58s (remain 4m 45s) Loss: 4.7408(5.2517) Grad: 41697.4570  
Epoch: [2][9

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [3][0/1348] Data 0.504 (0.504) Elapsed 0m 0s (remain 16m 7s) Loss: 5.6654(5.6654) Grad: inf  
Epoch: [3][100/1348] Data 0.500 (0.307) Elapsed 0m 52s (remain 10m 48s) Loss: 6.9765(5.2384) Grad: 47101.6562  
Epoch: [3][200/1348] Data 0.318 (0.310) Elapsed 1m 44s (remain 9m 58s) Loss: 4.6469(5.2254) Grad: 31249.5273  
Epoch: [3][300/1348] Data 0.281 (0.308) Elapsed 2m 36s (remain 9m 4s) Loss: 5.9456(5.1819) Grad: 26572.6641  
Epoch: [3][400/1348] Data 0.329 (0.309) Elapsed 3m 29s (remain 8m 14s) Loss: 4.4317(5.1181) Grad: 19016.8867  
Epoch: [3][500/1348] Data 0.300 (0.308) Elapsed 4m 20s (remain 7m 20s) Loss: 4.8906(5.0900) Grad: 24017.3691  
Epoch: [3][600/1348] Data 0.292 (0.310) Elapsed 5m 13s (remain 6m 29s) Loss: 5.4923(5.0604) Grad: 18708.3359  
Epoch: [3][700/1348] Data 0.300 (0.310) Elapsed 6m 6s (remain 5m 38s) Loss: 3.6025(4.9842) Grad: 15280.1123  
Epoch: [3][800/1348] Data 0.277 (0.310) Elapsed 6m 58s (remain 4m 45s) Loss: 5.3315(4.9276) Grad: 17394.8750  
Epoch: [3][9

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:972: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error ou

Epoch: [4][0/1348] Data 0.314 (0.314) Elapsed 0m 0s (remain 11m 43s) Loss: 3.0765(3.0765) Grad: inf  
Epoch: [4][100/1348] Data 0.283 (0.304) Elapsed 0m 52s (remain 10m 42s) Loss: 4.0362(2.9818) Grad: 39591.0664  
Epoch: [4][200/1348] Data 0.296 (0.305) Elapsed 1m 44s (remain 9m 53s) Loss: 3.1582(2.8102) Grad: 44106.3047  
Epoch: [4][300/1348] Data 0.286 (0.307) Elapsed 2m 36s (remain 9m 4s) Loss: 1.9992(2.7050) Grad: 39863.4688  
Epoch: [4][400/1348] Data 0.296 (0.306) Elapsed 3m 27s (remain 8m 10s) Loss: 1.8298(2.6637) Grad: 41702.4258  
Epoch: [4][500/1348] Data 0.318 (0.307) Elapsed 4m 20s (remain 7m 19s) Loss: 2.4465(2.6256) Grad: 53659.7109  
Epoch: [4][600/1348] Data 0.260 (0.306) Elapsed 5m 11s (remain 6m 27s) Loss: 1.3888(2.5901) Grad: 43884.8672  
Epoch: [4][700/1348] Data 0.328 (0.307) Elapsed 6m 4s (remain 5m 36s) Loss: 1.0368(2.5492) Grad: 46141.7070  
Epoch: [4][800/1348] Data 0.275 (0.308) Elapsed 6m 56s (remain 4m 44s) Loss: 1.3158(2.5349) Grad: 52639.6133  
Epoch: [4][

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.


Epoch: [5][0/1348] Data 0.306 (0.306) Elapsed 0m 0s (remain 11m 36s) Loss: 1.1596(1.1596) Grad: inf  
Epoch: [5][100/1348] Data 0.261 (0.309) Elapsed 0m 52s (remain 10m 50s) Loss: 0.2308(0.9449) Grad: 20387.1738  
Epoch: [5][200/1348] Data 0.288 (0.309) Elapsed 1m 44s (remain 9m 57s) Loss: 0.5297(0.8007) Grad: 36687.7031  
Epoch: [5][300/1348] Data 0.352 (0.309) Elapsed 2m 36s (remain 9m 5s) Loss: 0.0626(0.7349) Grad: 10205.4375  
Epoch: [5][400/1348] Data 0.306 (0.306) Elapsed 3m 27s (remain 8m 10s) Loss: 0.6695(0.6838) Grad: 39964.5273  
Epoch: [5][500/1348] Data 0.296 (0.306) Elapsed 4m 19s (remain 7m 19s) Loss: 0.5209(0.6353) Grad: 33711.8906  
Epoch: [5][600/1348] Data 0.289 (0.306) Elapsed 5m 11s (remain 6m 27s) Loss: 0.1626(0.6113) Grad: 17527.6895  
Epoch: [5][700/1348] Data 0.263 (0.305) Elapsed 6m 2s (remain 5m 34s) Loss: 0.3057(0.5902) Grad: 22306.9141  
Epoch: [5][800/1348] Data 0.293 (0.306) Elapsed 6m 54s (remain 4m 43s) Loss: 0.2006(0.5754) Grad: 21478.1465  
Epoch: [5][

In [22]:
! mkdir ./testholder
! cp -r ../input/humpback-whale-identification/test ./testholder/test

In [23]:
submission_img = datasets.ImageFolder('./testholder', transform=transform)
submission_loader = torch.utils.data.DataLoader(submission_img, batch_size=16)

In [24]:
filenames = []
for path, idx in submission_img.imgs:
    filenames.append(path[path.index('test/') + 5:])
filenames[:20]

['00028a005.jpg',
 '000dcf7d8.jpg',
 '000e7c7df.jpg',
 '0019c34f4.jpg',
 '001a4d292.jpg',
 '00247bc36.jpg',
 '0027089a4.jpg',
 '002de4d94.jpg',
 '002f52f0c.jpg',
 '002fd89d4.jpg',
 '00313e2d2.jpg',
 '00379666f.jpg',
 '0041a9867.jpg',
 '004344e9f.jpg',
 '0048970f9.jpg',
 '004fa8ff7.jpg',
 '00512687e.jpg',
 '006183fb4.jpg',
 '0061febfc.jpg',
 '0065d4964.jpg']

In [25]:
rev_lookup = {}
for key in allimg.class_to_idx:
    rev_lookup[allimg.class_to_idx[key]] = key
rev_lookup

{0: 'new_whale',
 1: 'w_0003639',
 2: 'w_0003c59',
 3: 'w_0027efa',
 4: 'w_00289b1',
 5: 'w_002c810',
 6: 'w_0032a46',
 7: 'w_003bae6',
 8: 'w_00656c0',
 9: 'w_0066399',
 10: 'w_007fefa',
 11: 'w_00904a7',
 12: 'w_009c9c5',
 13: 'w_00a41ba',
 14: 'w_00b3dc2',
 15: 'w_00d50c9',
 16: 'w_00d5466',
 17: 'w_00d5e98',
 18: 'w_00f340d',
 19: 'w_010f858',
 20: 'w_0115c24',
 21: 'w_0118bab',
 22: 'w_012678c',
 23: 'w_0135f5f',
 24: 'w_0140c92',
 25: 'w_015a4bf',
 26: 'w_016f0ea',
 27: 'w_0182687',
 28: 'w_0189b6d',
 29: 'w_01976db',
 30: 'w_01a1d88',
 31: 'w_01cb0c4',
 32: 'w_01cbbbd',
 33: 'w_01d790e',
 34: 'w_01e1b97',
 35: 'w_01e1fe0',
 36: 'w_01ed442',
 37: 'w_01f14e1',
 38: 'w_01fc429',
 39: 'w_02156b0',
 40: 'w_022b708',
 41: 'w_022d2f5',
 42: 'w_02469a1',
 43: 'w_02545ea',
 44: 'w_025911c',
 45: 'w_0277a07',
 46: 'w_027f528',
 47: 'w_0292e15',
 48: 'w_02aa597',
 49: 'w_02c7e9d',
 50: 'w_02d138d',
 51: 'w_02d5c46',
 52: 'w_02e8199',
 53: 'w_02ff6d6',
 54: 'w_0301302',
 55: 'w_030ac9b',
 5

In [26]:
all_preds = []
for step, (images, labels) in enumerate(submission_loader):
    images = images.to(device)
    with torch.no_grad():
        preds_by_image = m(images).argsort()
        for t in preds_by_image:
            # list top 1-5 best classes
            liked = t.tolist()[-5:][::-1]
            # convert class index to whale ID
            all_preds.append(list(map(lambda nid: rev_lookup[nid], liked)))
all_preds[:10]

[['new_whale', 'w_6caa59f', 'w_22d96e7', 'w_772c726', 'w_8b943ef'],
 ['new_whale', 'w_e3956f5', 'w_22d96e7', 'w_f5bf6df', 'w_8af65ed'],
 ['new_whale', 'w_a743e09', 'w_400791e', 'w_ae6ac74', 'w_c6c60c8'],
 ['w_9c506f6', 'w_a6703dd', 'w_59052ad', 'w_7938c79', 'w_d6aa3f3'],
 ['new_whale', 'w_92ac4b2', 'w_e145c7a', 'w_e2372d6', 'w_799b208'],
 ['new_whale', 'w_0e7ec27', 'w_1b6f30f', 'w_46e59f5', 'w_b4cf4b1'],
 ['w_b2bc0c9', 'w_8eae2c3', 'w_7e2eb3d', 'w_fe881f2', 'w_98bac23'],
 ['new_whale', 'w_b70c09f', 'w_50c4067', 'w_dd80742', 'w_1260eb5'],
 ['new_whale', 'w_715b1b5', 'w_f35e494', 'w_a13f1d0', 'w_a7e3d55'],
 ['new_whale', 'w_b60ef24', 'w_d09a047', 'w_6f74177', 'w_72f3685']]

In [27]:
! head -n 5 ../input/humpback-whale-identification/sample_submission.csv

Image,Id
00028a005.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
000dcf7d8.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
000e7c7df.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c
0019c34f4.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c


In [28]:
with open('submission.csv', 'w') as opfile:
    wrt = csv.writer(opfile)
    wrt.writerow(['Image', 'Id'])
    for idx in range(0, len(filenames)):
        wrt.writerow([
            filenames[idx],
            ' '.join(all_preds[idx]),
        ])

In [29]:
! head -n 5 submission.csv